In [1]:
import pandas as pd
import json

In [2]:
price_history = pd.read_json('price_collection.json')
price_history.columns = ['Date', 'Ticker', 'Volume', 'Open', 'Low', 'High', 'Close']
price_history.sort_values('Date', ascending=True, inplace=True)

with open('lotsize.json', "r") as read_file:
    lotsize = json.load(read_file)
    read_file.close()

black_list = ('DSKY','LNZL','LNZLP', 'ARSA', 'ASSB', 'AVAN', 'BISVP', 'BRZL', 'BSPBP', 'CHGZ', 'DZRD', 'DZRDP', 'EELT', 'ELTZ', 'GEMA', 'IGST', 'IGSTP', 'JNOS', 'JNOSP', 'KBSB', 'KCHE', 'KCHEP', 'KGKC', 'KGKCP', 'KMEZ', 'KOGK', 'KRKN', 'KROTP', 'KRSB', 'KRSBP', 'KTSB', 'KTSBP', 'KUBE', 'KUZB', 'LPSB', 'LVHK', 'MAGE', 'MAGEP', 'MFGS', 'MFGSP', 'MGNZ', 'MGTS', 'MISB', 'MISB', 'MISBP', 'MRKK', 'MRSB', 'NFAZ', 'NKSH', 'NNSB', 'NNSBP', 'OMZZP', 'PRMB', 'RBCM', 'RDRB', 'RGSS', 'ROSB', 'RTGZ', 'RTSB', 'RTSBP', 'RUSI', 'RZSB', 'SAGO', 'SAGOP', 'SARE', 'SAREP', 'SIBG', 'SLEN', 'STSB', 'STSBP', 'SVET', 'TASB', 'TASBP', 'TGKDP', 'TNSE', 'TORS', 'TORSP', 'TUZA', 'UCSS', 'UKUZ', 'URKZ', 'USBN', 'UTAR', 'VGSB', 'VGSBP', 'VJGZ', 'VJGZP', 'VLHZ', 'VRSBP', 'WTCM', 'WTCMP', 'YKEN', 'YKENP', 'YRSB', 'YRSBP', 'ZILL')

In [3]:
#prevent SettingWithCopyWarning message from appearing
pd.options.mode.chained_assignment = None
ranking = price_history.loc[price_history['Date'] == '2023-05-25']
for indx in ranking.index:
    ranking.at[indx, 'Lot'] = ranking.at[indx,'Volume'] / lotsize.get(ranking.at[indx,'Ticker'])
ranking.sort_values('Lot', ascending=True, inplace=True)
ranking['DecileRank']= pd.qcut(ranking['Lot'], q = 10, labels = False)

In [4]:
#FMBA ver4
def fmba(ema_days, lag_price, ATR_profit, ATR_loss, idea_end, idea_active, idea_ready):    
    for share in price_history.Ticker.unique():
        liquid = False        
        #select data for each ticker
        period = price_history[(price_history['Ticker'] == share) & (price_history['Date'] > '2022-11-01')]

        #searching high volume over 10 ema
        #create a short-list to count ema10 volume
        volume_check = period['Volume'].to_frame()
        volume_check['Volume_EMA10'] = volume_check['Volume'].ewm(span=10).mean()
        volume_check['increase'] = volume_check['Volume'] / volume_check['Volume_EMA10']

        #find all pump days high-volume min x2 over ema10 at green day mim and +8% price
        volume_check = volume_check [10:]
        pump_day = []
        for indx in volume_check.index:
            if volume_check.at[indx, 'increase'] > 2:
                if period.at[indx, 'Close'] > period.at[indx, 'Open'] * 1.08:
                    pump_day.append(indx)
        pump_day.sort

        #counting emas and ATR
        if len(pump_day) > 0:        
            ema_atr = period['Close'].to_frame()
            ema_atr['price_EMA'] = ema_atr['Close'].ewm(span=ema_days).mean()
            #counting ATR
            for i,tek in enumerate(ema_atr.index):
                if i == 0:
                    ema_atr.at[tek,'TR'] = period.at[tek,'High'] - period.at[tek,'Low']
                else:
                    pred = ema_atr.index[i-1]
                    tr1 = period.at[tek,'High'] - period.at[tek,'Low']
                    tr2 = abs(period.at[tek,'High'] - period.at[pred,'Close'])
                    tr3 = abs(period.at[pred,'Close'] - period.at[tek,'Low'])
                    ema_atr.at[tek,'TR'] = max(tr1, tr2, tr3)
            ema_atr['ATR'] = ema_atr['TR'].ewm(alpha=1/14).mean()

            #check strategy    
            share_active = False
            to_buy =  False
            prev_stop_loss = 0
            take_profit = 0
            stop_loss = 0
            for i,indx in enumerate(ema_atr.index):
                if indx in pump_day and not share_active:
                    to_buy = True
                    continue
                prev = ema_atr.index[i-1]
                if share_active:
                    take_profit = period.at[prev, 'Close'] + ATR_profit * ema_atr.at[prev, 'ATR']                
                    stop_loss = period.at[prev, 'Close'] + ATR_loss * ema_atr.at[prev, 'ATR']
                    stop_loss = max(stop_loss, prev_stop_loss, buy_price*1.01)
                    prev_stop_loss = stop_loss                    
                    if period.at[indx, 'Low'] <= stop_loss:
                        share_active = False
                        idea_res = round((stop_loss/buy_price-1)*100, 2)
                        idea_end.append([share, buy_date, period.at[indx, 'Date'], idea_res, liquid])
                    elif period.at[indx, 'High'] >= take_profit:
                        share_active = False
                        idea_res = round((take_profit/buy_price-1)*100, 2)
                        idea_end.append([share, buy_date, period.at[indx, 'Date'], idea_res, liquid])            
                else:
                    if to_buy: # replaced on previous date two times
                        if (period.at[indx, 'Low'] <= ema_atr.at[prev, 'price_EMA'] * lag_price ):
                            lot_trade = volume_check.at[indx, 'Volume_EMA10'] / lotsize.get(share)                    
                            if lot_trade > 10000:
                                liquid = True                        
                            buy_price = ema_atr.at[prev, 'price_EMA'] * lag_price
                            buy_date = period.at[indx, 'Date']
                            stop_loss = buy_price + ATR_loss * ema_atr.at[prev, 'ATR']
                            take_profit = buy_price + ATR_profit * ema_atr.at[prev, 'ATR']
                            share_active = True
                            to_buy = False
            if share_active:
                idea_res = round((ema_atr.at[indx, 'Close']/buy_price-1)*100, 2)
                idea_active.append([buy_date, share, idea_res, round(buy_price,5), round(take_profit,5), round(stop_loss,5), ranking.at[indx,'DecileRank'], liquid])
            else:
                if to_buy:
                    lot_trade = volume_check.at[indx, 'Volume_EMA10'] / lotsize.get(share)                
                    if lot_trade > 10000:
                        liquid = True
                    buy_price = ema_atr.at[indx, 'price_EMA'] + (2/(ema_days+1))*(period.at[indx,'Close']-ema_atr.at[indx, 'price_EMA'])
                    idea_ready.append([share, round(buy_price * lag_price , 5), ranking.at[indx,'DecileRank'],liquid])

In [9]:
def check_strategy (idea_end, black_list, scenary_result):    
    mas = [0.099, 0.124, 0.141, 0.165, 0.199, 0.249, 0.332]
    for posi in mas:
        res = 1000000
        depo = res
        depo_fact = res
        active_posi = {}
        period = price_history[price_history['Date'] > '2022-12-01']
        for d in period.Date.unique():
            for share in idea_end:
                if share[4]:
                    if share[2] == d:
                        tick_posi = active_posi.pop(share[0],0)            
                        depo += tick_posi * share[3]/100
                        depo_fact += tick_posi * (1+share[3]/100)
                    elif share[1] == d and share[0] not in black_list:
                        tick_posi = posi * depo
                        if tick_posi <= depo_fact:
                            depo_fact -= tick_posi
                            active_posi[share[0]] = tick_posi
#             if posi == 0.19:
#                 print(active_posi)
#         print(posi, round(depo_fact/res,2))
        sheets_count = str(int(1/posi))
        scenary_result[sheets_count].append(round(depo_fact/res,2))

In [5]:
idea_end = []
idea_active = []
idea_ready = []
fmba(10, 1.01, 1.5, -1, idea_end, idea_active, idea_ready) #fmba(ema_days, lag_price, ATR_profit, ATR_loss)
#print('Отработанные')                
#print(idea_end)
print('Активные')                
for i in idea_active:
    if i[1] not in black_list:
        print(i)
print('Ожидающие')
idea_ready.sort()
for j in range (9, -1, -1):
    for i in idea_ready:    
        if i[2] == j and i[0] not in black_list:
            print(i)

Активные
['2023-05-25', 'YNDX', 8.41, 2167.75747, 0, 0, 9, True]
['2023-05-25', 'SVAV', -0.48, 676.73547, 428.32882, 382.50728, 5, True]
Ожидающие
['ETLN', 75.89107, 9, True]
['BANEP', 1432.62657, 8, True]
['BANE', 1643.6562, 7, True]
['FESH', 51.76661, 7, True]
['TGKBP', 0.01169, 7, True]
['GAZA', 785.5202, 6, False]
['GAZAP', 482.9125, 5, False]
['TGKN', 0.00892, 4, True]
['KRKOP', 16.91928, 3, True]


In [14]:
arr_ema_days = (20, 10)
arr_atr_loss = (-0.5, -1)
arr_lag_price = (0.98, 0.985, 0.99, 0.995, 1, 1.005, 1.01, 1.015, 1.02)
scenary_result = {'EMA_days':[], 'ATR_loss':[], 'lag_price':[], '3':[], '4':[], '5':[], '6':[], '7':[], '8':[], '10':[]}
for i_ema_days in arr_ema_days:
    for i_ATR_loss in arr_atr_loss:
        for i_lag_price in arr_lag_price:
            idea_end = []
            idea_active = []
            idea_ready = []
            scenary_result['EMA_days'].append(i_ema_days)
            scenary_result['ATR_loss'].append(i_ATR_loss)
            scenary_result['lag_price'].append(i_lag_price-1)
            fmba(i_ema_days, i_lag_price, 1.5, i_ATR_loss, idea_end, idea_active, idea_ready) 
            check_strategy (idea_end, black_list, scenary_result)
df_res = pd.DataFrame(scenary_result)

In [15]:
df_res

,EMA_days,ATR_loss,lag_price,3,4,5,6,7,8,10
0,20,-0.5,-0.020,1.28,1.21,1.18,1.15,1.13,1.12,1.10
1,20,-0.5,-0.015,1.16,1.13,1.11,1.10,1.09,1.08,1.07
2,20,-0.5,-0.010,1.19,1.17,1.14,1.12,1.11,1.10,1.08
3,20,-0.5,-0.005,1.29,1.22,1.20,1.16,1.14,1.13,1.10
4,20,-0.5,0.000,1.33,1.27,1.24,1.20,1.17,1.15,1.12
5,20,-0.5,0.005,1.28,1.26,1.23,1.20,1.18,1.16,1.14
6,20,-0.5,0.010,1.41,1.32,1.29,1.26,1.23,1.20,1.18
7,20,-0.5,0.015,2.03,1.77,1.60,1.49,1.42,1.37,1.30
8,20,-0.5,0.020,1.90,1.67,1.52,1.43,1.37,1.34,1.26
9,20,-1.0,-0.020,1.29,1.22,1.18,1.16,1.14,1.12,1.10
